In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import sklearn 
import string
from IPython.display import display, Latex, Markdown

# Classifying tweets

In this problem, you will be analyzing Twitter data extracted using [this](https://dev.twitter.com/overview/api) api. The data contains tweets posted by the following six Twitter accounts: `realDonaldTrump, mike_pence, GOP, HillaryClinton, timkaine, TheDemocrats`

For every tweet, there are two pieces of information:
- `screen_name`: the Twitter handle of the user tweeting and
- `text`: the content of the tweet.

The tweets have been divided into two parts - train and test available to you in CSV files. For train, both the `screen_name` and `text` attributes were provided but for test, `screen_name` is hidden.

The overarching goal of the problem is to "predict" the political inclination (Republican/Democratic) of the Twitter user from one of his/her tweets. The ground truth (i.e., true class labels) is determined from the `screen_name` of the tweet as follows
- `realDonaldTrump, mike_pence, GOP` are Republicans
- `HillaryClinton, timkaine, TheDemocrats` are Democrats

Thus, this is a binary classification problem. 

The problem proceeds in three stages:
- **Text processing (25%)**: We will clean up the raw tweet text using the various functions offered by the [nltk](http://www.nltk.org/genindex.html) package.
- **Feature construction (25%)**: In this part, we will construct bag-of-words feature vectors and training labels from the processed text of tweets and the `screen_name` columns respectively.
- **Classification (50%)**: Using the features derived, we will use [sklearn](http://scikit-learn.org/stable/modules/classes.html) package to learn a model which classifies the tweets as desired. 

You will use two new python packages in this problem: `nltk` and `sklearn`, both of which should be available with anaconda. However, NLTK comes with many corpora, toy grammars, trained models, etc, which have to be downloaded manually. This assignment requires NLTK's stopwords list and WordNetLemmatizer. Install them using:

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
# Verify that the following commands work for you, before moving on.

lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bodav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's begin!

## A. Text Processing 

You first task to fill in the following function which processes and tokenizes raw text. The generated list of tokens should meet the following specifications:
1. The tokens must all be in lower case.
2. The tokens should appear in the same order as in the raw text.
3. The tokens must be in their lemmatized form. If a word cannot be lemmatized (i.e, you get an exception), simply catch it and ignore it. These words will not appear in the token list.
4. The tokens must not contain any punctuations. Punctuations should be handled as follows: (a) Apostrophe of the form `'s` must be ignored. e.g., `She's` becomes `she`. (b) Other apostrophes should be omitted. e.g, `don't` becomes `dont`. (c) Words must be broken at the hyphen and other punctuations. 
5. The tokens must not contain any part of a url.

Part of your work is to figure out a logical order to carry out the above operations. You may find `string.punctuation` useful, to get hold of all punctuation symbols. Look for [regular expressions](https://docs.python.org/3/library/re.html) capturing urls in the text. Your tokens must be of type `str`. Use `nltk.word_tokenize()` for tokenization once you have handled punctuation in the manner specified above. 

You would want to take a look at the `lemmatize()` function [here](https://www.nltk.org/_modules/nltk/stem/wordnet.html).
In order for `lemmatize()` to give you the root form for any word, you have to provide the context in which you want to lemmatize through the `pos` parameter: `lemmatizer.lemmatize(word, pos=SOMEVALUE)`. The context should be the part of speech (POS) for that word. The good news is you do not have to manually write out the lexical categories for each word because [nltk.pos_tag()](https://www.nltk.org/book/ch05.html) will do this for you. Now you just need to use the results from `pos_tag()` for the `pos` parameter.
However, you can notice the POS tag returned from `pos_tag()` is in different format than the expected pos by `lemmatizer`.
> pos
(Syntactic category): n for noun files, v for verb files, a for adjective files, r for adverb files.

You need to map these pos appropriately. `nltk.help.upenn_tagset()` provides description of each tag returned by `pos_tag()`.

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>
 

In [3]:
from nltk.tokenize import word_tokenize
import re
def remove_punctuation(value):
    result = ""
    for c in value:
        if c not in string.punctuation:
            result += c
        else:
            result +=" "
    return result
def posMapping(s):
    posMapping = {"N":'n',
                  "V":'v',
                  "J":'a',
                  "R":'r'}
    return posMapping.get(s,'n')
def lem(oo):
    lemmatized_output = [lemmatizer.lemmatize(oo)]
    return lemmatized_output 

"""def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    print(tag)
    posMapping = {"N":'n',
                  "V":'v',
                  "J":'a',
                  "R":'r'}
    return posMapping.get(tag,'n')"""

def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ Normalizes case and handles punctuation
    Inputs:
        text: str: raw text
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs:
        list(str): tokenized text
    """
    
    text = text.replace("-"," ")
    text = text.replace("'s","")
    text = text.replace("'","")
    #print(text)
    url_pattern = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?'
    #text_1 = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    #text_1 = re.sub(r"(http|https)\S+", "", text)
    text_1 = re.sub(url_pattern, "", text)
    input_str = text_1.lower()
    x = remove_punctuation(input_str)
    #print(text)
    y = nltk.word_tokenize(x) 
    #print(y)
    z = nltk.pos_tag(y)
    #print(z)
    e = []
    for lis in z:
        res = lis[1][0]
#res = [lis[1][0] for lis in z] 
    #print(res)
        s= posMapping(res)
        e.append(s)
    #print(e)
    t = []
    for i in range(len(y)):
        p = lemmatizer.lemmatize(y[i],e[i])
        t.append(p)
    #print(p)
    return t
    
    """"p = lemmatizer.lemmatize('played',get_wordnet_pos('played'))
    #print(p)
    lemmatized_output = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in y]
    #print(get_wordnet_pos('played'))
    #return lemmatized_output"""
    

You can test the above function as follows. Try to make your test strings as exhaustive as possible. Some checks are:

In [4]:
print(process("I'm doing well! How about you?"))
# ['im', 'do', 'well', 'how', 'about', 'you']

print(process("Education is the ability to listen to almost anything without losing your temper or your self-confidence."))
# ['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']

print(process("been had done languages cities mice"))
# ['be', 'have', 'do', 'language', 'city', 'mice']


print(process("It's hilarious. Check it out https://t.co/dummyurl"))
# ['it', 'hilarious', 'check', 'it', 'out']

['im', 'do', 'well', 'how', 'about', 'you']
['education', 'be', 'the', 'ability', 'to', 'listen', 'to', 'almost', 'anything', 'without', 'lose', 'your', 'temper', 'or', 'your', 'self', 'confidence']
['be', 'have', 'do', 'language', 'city', 'mice']
['it', 'hilarious', 'check', 'it', 'out']


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br> 

You will now use the `process()` function we implemented to convert the pandas dataframe we just loaded from tweets_train.csv file. Your function should be able to handle any data frame which contains a column called `text`. The data frame you return should replace every string in `text` with the result of `process()` and retain all other columns as such. Do not change the order of rows/columns. Before writing `process_all()`, load the data into a DataFrame and look at its format:

In [5]:
tweets = pd.read_csv("tweets_train.csv", na_filter=False)
print(tweets.head())

      screen_name                                               text
0             GOP  RT @GOPconvention: #Oregon votes today. That m...
1    TheDemocrats  RT @DWStweets: The choice for 2016 is clear: W...
2  HillaryClinton  Trump's calling for trillion dollar tax cuts f...
3  HillaryClinton  .@TimKaine's guiding principle: the belief tha...
4        timkaine  Glad the Senate could pass a #THUD / MilCon / ...


In [6]:
def process_all(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    """ process all text in the dataframe using process() function.
    Inputs
        df: pd.DataFrame: dataframe containing a column 'text' loaded from the CSV file
        lemmatizer: an instance of a class implementing the lemmatize() method
                    (the default argument is of type nltk.stem.wordnet.WordNetLemmatizer)
    Outputs
        pd.DataFrame: dataframe in which the values of text column have been changed from str to list(str),
                        the output from process() function. Other columns are unaffected.
    """
    #[YOUR CODE HERE]
    df["text"] = df["text"].apply(lambda text : process(text))
    return df
    
    """out = []
    text = df.iloc[:,1]
    #print(text)
    for i in text:
        ans = process(i)
        out.append(ans)
        
    y = df
    y.drop(columns=['text'],inplace = True)
    y['text'] = out
    #print(tweets)
    return y"""

In [7]:
# test your code
processed_tweets = process_all(tweets.copy())
print(processed_tweets.head())

#       screen_name                                               text
# 0             GOP  [rt, gopconvention, oregon, vote, today, that,...
# 1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
# 2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
# 3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
# 4        timkaine  [glad, the, senate, could, pass, a, thud, milc...

      screen_name                                               text
0             GOP  [rt, gopconvention, oregon, vote, today, that,...
1    TheDemocrats  [rt, dwstweets, the, choice, for, 2016, be, cl...
2  HillaryClinton  [trump, call, for, trillion, dollar, tax, cut,...
3  HillaryClinton  [timkaine, guide, principle, the, belief, that...
4        timkaine  [glad, the, senate, could, pass, a, thud, milc...


## B. Feature Construction 

The next step is to derive feature vectors from the tokenized tweets. In this section, you will be constructing a bag-of-words TF-IDF feature vector. But before that, as you may have guessed, the number of possible words is prohibitively large and not all of them may be useful for our classification task. We need to determine which words to retain, and which to omit. A common heuristic is to construct a frequency distribution of words in the corpus and prune out the head and tail of the distribution. The intuition of the above operation is as follows. Very common words (i.e. stopwords) add almost no information regarding similarity of two pieces of text. Similarly with very rare words. NLTK has a list of in-built stop words which is a good substitute for head of the distribution. We will consider a word rare if it occurs only in a single document (row) in whole of `tweets_train.csv`. 

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br> 

Construct a sparse matrix of features for each tweet with the help of `sklearn.feature_extraction.text.TfidfVectorizer` (documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). You need to pass a parameter `min_df=2` to filter out the words occuring only in one document in the whole training set. Remember to ignore the stop words as well. You must leave other optional parameters (e.g., `vocab`, `norm`, etc) at their default values. 

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
def identity_tokenizer(text):
    print(text)
    return text
def create_features(processed_tweets, stop_words):
    """ creates the feature matrix using the processed tweet text
    Inputs:
        tweets: pd.DataFrame: tweets read from train/test csv file, containing the column 'text'
        stop_words: list(str): stop_words by nltk stopwords
    Outputs:
        sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used
            we need this to tranform test tweets in the same way as train tweets
        scipy.sparse.csr.csr_matrix: sparse bag-of-words TF-IDF feature matrix
    """
    #[your code here
    from nltk.corpus import stopwords
    vectorizer = TfidfVectorizer(lowercase = False, stop_words=stop_words, min_df = 2,)
    #vectorizer = TfidfVectorizer(lowercase = False, stop_words=stop_words,tokenizer = identity_tokenizer, min_df = 2,)
    corpus = []
    processed_tweets["text"].apply(lambda text : corpus.append(text))
    #print(corpus)
    listoflists=corpus
    corpus=[" ".join(i) for i in listoflists]
    #print(corpus)
    Feature =vectorizer.fit_transform(corpus)
    #Feature.toarray()
    Y = vectorizer.get_feature_names()
    #print(Y)
    return(vectorizer,Feature)

In [9]:
# execute this code 
(tfidf, X) = create_features(processed_tweets, stopwords)

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>


Also for each tweet, assign a class label (0 or 1) using its `screen_name`. Use 0 for realDonaldTrump, mike_pence, GOP and 1 for the rest.

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def class_labels(s):
    if s == "realDonaldTrump" :
        return 0
    elif s == "mike_pence" :
        return 0
    elif s == "GOP":
        return 0
    else:
        return 1

def create_labels(processed_tweets):
    """ creates the class labels from screen_name
    Inputs:
        tweets: pd.DataFrame: tweets read from train file, containing the column 'screen_name'
    Outputs:
        numpy.ndarray(int): dense binary numpy array of class labels
    """
    processed_tweets = processed_tweets['screen_name'].apply(lambda screen_name: class_labels(screen_name))
    #print(processed_tweets)
    return np.array(processed_tweets,dtype = int)

In [11]:
# execute this code
y = create_labels(processed_tweets)

## C. Classification 

And finally, we are ready to put things together and learn a model for the classification of tweets. The classifier you will be using is [`sklearn.svm.SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (Support Vector Machine). 

At the heart of SVMs is the concept of kernel functions, which determines how the similarity/distance between two data points in computed. `sklearn`'s SVM provides four kernel functions: `linear`, `poly`, `rbf`, `sigmoid` (details [here](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)) but you can also implement your own distance function and pass it as an argument to the classifier.

Through the various functions you implement in this part, you will be able to learn a classifier, score a classifier based on how well it performs, use it for prediction tasks and compare it to a baseline.

Specifically, you will carry out the following tasks (Q5-9) in order:

1. Implement and evaluate a simple baseline classifier MajorityLabelClassifier.
2. Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}. 
3. Implement the `evaluate_classifier()` function which scores a classifier based on accuracy of a given dataset.
4. Implement `best_model_selection()` to perform cross-validation by calling `learn_classifier()` and `evaluate_classifier()` for different folds and determine which of the four kernels performs the best.
5. Go back to `learn_classifier()` and fill in the best kernel. 

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>
 

To determine whether your classifier is performing well, you need to compare it to a baseline classifier. A baseline is generally a simple or trivial classifier and your classifier should beat the baseline in terms of a performance measure such as accuracy. Implement a classifier called `MajorityLabelClassifier` that always predicts the class equal to **mode** of the labels (i.e., the most frequent label) in training data. Part of the code is done for you. Implement the `fit` and `predict` methods. Initialize your classifier appropriately.

In [12]:
from scipy import stats
from scipy.sparse import csr_matrix
class MajorityLabelClassifier():
  """
  A classifier that predicts the mode of training labels
  """
  def __init__(self):
    """
    Initialize
    """
    #[YOUR CODE HERE]
    self.mode = 0
  
  def fit(self, X, y):
        
        
        """
         Implement fit by taking training data X and their labels y and finding the mode of y
        """
        #[YOUR CODE HERE]
        first_label = 0
        second_label = 0
        v1 = y[0]
        v2 = None
        for i in y:
            if i != v1:
                v2 = i
                break
        for i in y:
            if i == v1:
                first_label+=1
            else:
                second_label+=1
                
        if first_label>second_label:
            self.mode = v1
        else:
            self.mode = v2
        
    
  def predict(self, X):
    
    """
    Implement to give the mode of training labels as a prediction for each data instance in X
    return labels
    """
    #[YOUR CODE HERE]
    result = []
    for j in X:
        result.append(self.mode)
    return result

    


# Report the accuracy of your classifier by comparing the predicted label of each example to its true label
#[YOUR CODE HERE]
count = 0
from sklearn.metrics import accuracy_score
X1 =  [i for i in processed_tweets['text']]
Base_Line_Classifier = MajorityLabelClassifier()
Base_Line_Classifier.fit(X1,y)
predicted_labels = Base_Line_Classifier.predict(X1)
for j in range(y.size):
    if y[j] == predicted_labels[j]:
        count+=1
print(count/y.size)
# training accuracy = 0.500173

0.5001734304543878


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>



Implement the `learn_classifier()` function assuming `kernel` is always one of {`linear`, `poly`, `rbf`, `sigmoid`}. Stick to default values for any other optional parameters.

In [13]:
def learn_classifier(X_train, y_train, kernel):
    """ learns a classifier from the input features and labels using the kernel function supplied
    Inputs:
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features, output of create_features()
        y_train: numpy.ndarray(int): dense binary vector of class labels, output of create_labels()
        kernel: str: kernel function to be used with classifier. [linear|poly|rbf|sigmoid]
    Outputs:
        sklearn.svm.classes.SVC: classifier learnt from data
    """
    
    #[YOUR CODE HERE]
    from sklearn import svm
    from sklearn.svm import SVC
    clf = svm.SVC(kernel = kernel,gamma='scale')
    return clf.fit(X_train,y_train)

In [14]:
# execute code
classifier = learn_classifier(X, y, 'linear')
#classifier

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>
 

Now that we know how to learn a classifier, the next step is to evaluate it, ie., characterize how good its classification performance is. This step is necessary to select the best model among a given set of models, or even tune hyperparameters for a given model.

There are two questions that should now come to your mind:
1. **What data to use?** 
    - **Validation Data**: The data used to evaluate a classifier is called **validation data** (or hold-out data), and it is usually different from the data used for training. The model or hyperparameter with the best performance in the held out data is chosen. This approach is relatively fast and simple but vulnerable to biases found in validation set.
    - **Cross-validation**: This approach divides the dataset in $k$ groups (so, called k-fold cross-validation). One of group is used as test set for evaluation and other groups as training set. The model or hyperparameter with the best average performance across all k folds is chosen. For this question you will perform 4-fold cross validation to determine the best kernel. We will keep all other hyperparameters default for now. This approach provides robustness toward biasness in validation set. However, it takes more time.
    
2. **And what metric?** There are several evaluation measures available in the literature (e.g., accuracy, precision, recall, F-1,etc) and different fields have different preferences for specific metrics due to different goals. We will go with accuracy. According to wiki, **accuracy** of a classifier measures the fraction of all data points that are correctly classified by it; it is the ratio of the number of correct classifications to the total number of (correct and incorrect) classifications. `sklearn.metrics` provides a number of performance metrics.

Now, implement the following function.

In [15]:
from sklearn.metrics import accuracy_score
def evaluate_classifier(classifier, X_validation, y_validation):
    """ evaluates a classifier based on a supplied validation data
    Inputs:
        classifier: sklearn.svm.classes.SVC: classifer to evaluate
        X_train: scipy.sparse.csr.csr_matrix: sparse matrix of features
        y_train: numpy.ndarray(int): dense binary vector of class labels
    Outputs:
        double: accuracy of classifier on the validation data
    """
    #[YOUR CODE HERE]
    """from sklearn.model_selection import cross_validate
    from sklearn.metrics import accuracy_score
    scores = cross_validate(classifier, X_validation, y_validation, cv=4,scoring = 'accuracy')
    return scores"""
    predicted_values = classifier.predict(X_validation)
    actual_values = y_validation
    return(accuracy_score(predicted_values,actual_values))

In [16]:
# test your code by evaluating the accuracy on the training data
accuracy = evaluate_classifier(classifier, X, y)
print(accuracy) 
# should give 0.956700196554515

0.9520753844375073


<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br><br>
 

Now it is time to decide which kernel works best by using the cross-validation technique. Write code to split the training data into 4-folds (75% training and 25% validation) by shuffling randomly. For each kernel, record the average accuracy for all folds and determine the best classifier. Since our dataset is balanced (both classes are in almost equal propertion), `sklearn.model_selection.KFold` [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) can be used for cross-validation.

In [17]:
kf = sklearn.model_selection.KFold(n_splits=4, random_state=1, shuffle=True,)
#kf
#print(tfidf)

Then use the following code to determine which classifier is the best. 

In [18]:
from statistics import mean
def best_model_selection(kf, X, y):
    """
      Select the kernel giving best results using k-fold cross-validation.
      Other parameters should be left default.
      Input:
        kf (sklearn.model_selection.KFold): kf object defined above
        X (scipy.sparse.csr.csr_matrix): training data
        y (array(int)): training labels
      Return:
        best_kernel (string)
    """
      #[YOUR CODE HERE]
    #best_model = []
    actual_list = []
    for kernel in ['linear', 'rbf', 'poly', 'sigmoid']:
        list = []
    #[YOUR CODE HERE] 
    # Use the documentation of KFold cross-validation to split ..
    # training data and test data from create_features() and create_labels()
    # call learn_classifer() using training split of kth fold
    # evaluate on the test split of kth fold
    # record avg accuracies and determine best model (kernel)
    #return best kernel as string
        for train_index, test_index in kf.split(X):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            classifier_1 = learn_classifier(X_train, y_train, kernel)
            accuracy_1 = evaluate_classifier(classifier_1, X_test, y_test)
            list.append(accuracy_1)
        actual_list.append((kernel,mean(list)))
    #print(actual_list)
    z = max(actual_list,key = lambda x:x[1])[0]
    #print(z)
    return(z)  
#Test your code
best_kernel = best_model_selection(kf, X, y)
#best_kernel

<img src="bikeshare.png" width="100px" align="left" float="left"/>
<br><br>  

We're almost done! It's time to write a nice little wrapper function that will use our model to classify unlabeled tweets from tweets_test.csv file. 

In [19]:
def classify_tweets(tfidf, classifier, unlabeled_tweets):
    """ predicts class labels for raw tweet text
    Inputs:
        tfidf: sklearn.feature_extraction.text.TfidfVectorizer: the TfidfVectorizer object used on training data
        classifier: sklearn.svm.classes.SVC: classifier learnt
        unlabeled_tweets: pd.DataFrame: tweets read from tweets_test.csv
    Outputs:
        numpy.ndarray(int): dense binary vector of class labels for unlabeled tweets
    """
    #[YOUR CODE HERE]
    processed_tweets_1 = process_all(unlabeled_tweets)
    corpus_1 = []
    processed_tweets_1["text"].apply(lambda text : corpus_1.append(text))
    #print(corpus)
    listoflists=corpus_1
    corpus_1 = [" ".join(i) for i in listoflists]
    Feature = tfidf.transform(corpus_1)
    predicted_values_1 = classifier.predict(Feature)
    return predicted_values_1
    

In [20]:
# Fill in best classifier in your function and re-trian your classifier using all training data
# Get predictions for unlabelled test data
classifier = learn_classifier(X, y, best_kernel)
unlabeled_tweets = pd.read_csv("tweets_test.csv", na_filter=False)
y_pred = classify_tweets(tfidf, classifier, unlabeled_tweets)

SVM classifier performs better than the baseline classifier, because the accuracy of baseline classifier is 50% approximately, however the accuracy of SVM classifier is 95.6%.

In baseline classifier if we give inputs of different values, we are predicting the same output for every input which is mode, but this is not the case with the SVM classifier.  